In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install pythainlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 77.4 MB/s eta 0:00:00:00:0100:01


In [4]:
list_zer0 = ["แถม", "up", "both", "phone", "free", "โปร", "ฟรี", "บาท", "sale", "ลด"]
list_one1 = ["คอร์ส", "โปร", "trial", "sale", "ราคา", "data", "ฟรี", "บาท", "grabfood", "ลด"]
total_words = list(set(list_zer0 + list_one1))
dict_word = {
    "0": list_zer0,
    "1": list_one1
}

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

In [19]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="kornwtp/ThaiBERT-Small")

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/115M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/176 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/573k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("kornwtp/ThaiBERT-Small")
model = AutoModelForMaskedLM.from_pretrained("kornwtp/ThaiBERT-Small")

In [ ]:
def analyze_sentiment(word):
    inputs = tokenizer(word, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # ระดับ sentiment (1-5, 1=negative, 5=positive)
    rating = torch.argmax(probabilities).item() + 1
    confidence = torch.max(probabilities).item()
    
    return {
        "word": word,
        "sentiment": "positive" if rating >= 4 else "negative" if rating <= 2 else "neutral",
        "score": confidence,
        "rating": rating
    }

In [ ]:
results = []
for word in total_words:
    results.append(analyze_sentiment(word))

# เรียงตามคะแนน sentiment
sorted_results = sorted(results, key=lambda x: x["rating"], reverse=True)

In [38]:
sorted_results

[{'word': 'grabfood',
  'sentiment': 'positive',
  'score': 0.9107881784439087,
  'rating': 92632},
 {'word': 'both',
  'sentiment': 'positive',
  'score': 0.9057531356811523,
  'rating': 65209},
 {'word': 'sale',
  'sentiment': 'positive',
  'score': 0.878376305103302,
  'rating': 62111},
 {'word': 'คอร์ส',
  'sentiment': 'positive',
  'score': 0.971721887588501,
  'rating': 62071},
 {'word': 'data',
  'sentiment': 'positive',
  'score': 0.848181962966919,
  'rating': 58788},
 {'word': 'ฟรี',
  'sentiment': 'positive',
  'score': 0.9288156628608704,
  'rating': 46835},
 {'word': 'free',
  'sentiment': 'positive',
  'score': 0.8652098774909973,
  'rating': 45705},
 {'word': 'trial',
  'sentiment': 'positive',
  'score': 0.8720271587371826,
  'rating': 43940},
 {'word': 'up',
  'sentiment': 'positive',
  'score': 0.9166219830513,
  'rating': 40189},
 {'word': 'phone',
  'sentiment': 'positive',
  'score': 0.8450317978858948,
  'rating': 36876},
 {'word': 'แถม',
  'sentiment': 'positive'

In [ ]:
import plotly.express as px

from pythainlp.word_vector import WordVector
from sklearn.decomposition import PCA

In [ ]:
wv = WordVector()

In [ ]:
def get_vector(word):
    return wv.get_model().get_vector(word)

In [ ]:
get_vector("trial")

In [ ]:
from itertools import combinations

# Cosine Similarity

In [ ]:
for w1, w2 in combinations(list_zer0, 2):
    try:
        sim = wv.similarity(w1, w2)
        print(f"{w1} - {w2} : {sim:.2f}")
    except KeyError:
        print(f"ข้าม: {w1}, {w2} ไม่มีในเวกเตอร์")

# Comparison words cross clusters

In [ ]:
for w0 in list_zer0:
    for w1 in list_one1:
        try:
            sim = wv.similarity(w0, w1)
            if sim > 0.7:
                print(f"{w0} ↔ {w1} : {sim:.2f}")
        except KeyError:
            continue  

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
words = list_zer0 + list_one1

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
words = list_zer0 + list_one1

In [ ]:
import numpy as np

In [ ]:
vectors = []
filtered_words = []

In [ ]:
for word in words:
    try:
        vec = get_vector(word)
    except KeyError:
        vec = np.random.rand(300)  
        vec = vec / np.linalg.norm(vec) 
    vectors.append(vec)
    filtered_words.append(word)

In [ ]:
len(words), len(vectors), len(filtered_words)

In [ ]:
import pandas as pd

In [ ]:
pca = PCA(n_components=3)
points = pca.fit_transform(vectors)

df_plot = pd.DataFrame({
    "word": words,
    "x": points[:, 0],
    "y": points[:, 1],
    "z": points[:, 2]
})

In [ ]:
fig = px.scatter(
    df_plot,
    x="x",
    y="y",
    text="word",
    hover_name="word",
    title="Word Embedding (PCA Projection)",
    width=800,
    height=500  
)
fig.update_traces(textposition="top center", marker=dict(size=10))
fig.update_layout(showlegend=False)
# plt.savefig('ddd.png')
fig.show()

In [ ]:
fig = px.scatter_3d(
    df_plot,
    x="x",
    y="y",
    z="z",              # เพิ่มแกน z ด้วย
    text="word",
    hover_name="word",
    title="Word Embedding (PCA 3D Projection)",
    width=800,
    height=600
)

fig.update_traces(marker=dict(size=5))
fig.update_layout(showlegend=False)

fig.show()